In [17]:
import settings
import helpers
import sys
import os
import glob
import random
import pandas
import ntpath
import cv2
import numpy as np
from tensorflow.keras import backend as K
import math
import pydicom as dicom
from PIL import Image, ImageDraw

In [2]:
import tensorflow as tf
from tensorflow.compat.v1.keras.backend import set_session
import Train
config = tf.compat.v1.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
set_session(tf.compat.v1.Session(config=config))

In [55]:
pred_labels = pandas.read_csv('F:/Cengiz/Nodules-Detection/prediction/predictions10_luna16_fs/1.3.6.1.4.1.14519.5.2.1.6279.6001.330643702676971528301859647742.csv')
pred_labels.head()

,anno_index,coord_x,coord_y,coord_z,nodule_chance,status
0,0,0.1444,0.4444,0.6125,0.6562,FP
1,1,0.1778,0.4444,0.6125,0.6142,FP
2,2,0.4778,0.2778,0.6875,0.7235,FP
3,3,0.4778,0.3111,0.6875,0.6820,FP
4,4,0.5444,0.5778,0.6875,0.6290,FP


In [8]:
def load_patient(src_dir):
    # slices = [dicom.read_file(src_dir + '/' + s) for s in os.listdir(src_dir)]
    slices = []
    print(src_dir)
    for files in glob.glob(src_dir + "/*.dcm"):
        slices.append(dicom.read_file(files))
    print(len(slices))
    slices.sort(key=lambda x: int(x.InstanceNumber))
    try:
        slice_thickness = numpy.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = numpy.abs(slices[0].SliceLocation - slices[1].SliceLocation)
    if slice_thickness == 0:
        slice_thickness = numpy.abs(slices[4].ImagePositionPatient[2] - slices[5].ImagePositionPatient[2])
    for s in slices:
        s.SliceThickness = slice_thickness
    return slices
def get_pixels_hu(slices):
    image = numpy.stack([s.pixel_array for s in slices])
    image = image.astype(float)
    for slice_number in range(len(slices)):
        intercept = slices[slice_number].RescaleIntercept
        slope = slices[slice_number].RescaleSlope
        if slope != 1:
            image[slice_number] = slope * image[slice_number].astype(numpy.float64)
            image[slice_number] = image[slice_number]
        image[slice_number] += intercept

    return numpy.array(image, dtype=numpy.float64)


In [77]:
INPUT_DICOM_PATH = 'D:/LIDC-IDRI/LIDC-IDRI-0802/1.3.6.1.4.1.14519.5.2.1.6279.6001.322126192251489550021873181090/1.3.6.1.4.1.14519.5.2.1.6279.6001.330643702676971528301859647742/'
slices = load_patient(INPUT_DICOM_PATH)
pixels = get_pixels_hu(slices)
image = pixels
shape_image = image.shape
nodules_coord = []
slices_nod_num = []
for index, row in pred_labels.iterrows():
    coord_z = row['coord_z'] * shape_image[0]
    coord_z = int(round(coord_z))
    coord_y = row['coord_y'] * shape_image[2]
    coord_y = int(round(coord_y))
    coord_x = row['coord_x'] * shape_image[1]
    coord_x = int(round(coord_x))
    nodules_coord.append([coord_x,coord_y,coord_z])
    slices_nod_num.append(coord_z)
print(nodules_coord)

D:/LIDC-IDRI/LIDC-IDRI-0802/1.3.6.1.4.1.14519.5.2.1.6279.6001.322126192251489550021873181090/1.3.6.1.4.1.14519.5.2.1.6279.6001.330643702676971528301859647742/
256
[[74, 228, 157], [91, 228, 157], [245, 142, 176], [245, 159, 176], [279, 296, 176], [296, 296, 176], [245, 142, 186], [245, 159, 186], [125, 330, 195], [364, 193, 214], [159, 210, 214], [364, 210, 214], [142, 228, 214], [159, 228, 214], [125, 262, 214], [142, 262, 214], [125, 279, 214], [142, 279, 214], [364, 193, 224], [159, 210, 224], [142, 228, 224], [159, 228, 224], [125, 262, 224], [142, 262, 224], [125, 279, 224], [142, 279, 224]]


In [62]:
print(pixel_spacing,image.shape)

[0.703125, 0.703125, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1

In [70]:
def apply_ct_window(img, window):
    # window = (window width, window level)
    R = (img-window[1]+0.5*window[0])/window[0]
    R[R<0] = 0
    R[R>1] = 1
    return R
def normalize(image):
    MIN_BOUND = -1000.0
    MAX_BOUND = 400.0
    image = (image - MIN_BOUND) / (MAX_BOUND - MIN_BOUND)
    image[image > 1] = 1.
    image[image < 0] = 0.
    return image
#img = image[0]
#print(img.shape)
#display_img = apply_ct_window(img, [400,-1000])
#display_img = normalize(img)
#ds = slices[0]

In [78]:
for count,slice_ in enumerate(slices):
    #display_img = normalize(img)
    pixels = get_pixels_hu([slice_])
    image = pixels
    img = image[0]
    #print(img.shape)
    ds = slice_
    normalized_image = normalize(img)
    if count in slices_nod_num:
        for coord in nodules_coord:
            if count == coord[2]:
                top, left, bottom, right = [coord[1]-15,coord[0]-15,coord[1]+15,coord[0]+15]
                #top, left, bottom, right = [211,99,291,158]
                thickness = 2
                img_bbox = Image.fromarray((255*normalized_image).astype('uint8'))
                img_bbox = img_bbox.convert('RGB')
                #rect = Image.new ('RGB', img_bbox.size)
                draw = ImageDraw.Draw(img_bbox)
                for i in range(thickness):
                    draw.rectangle(
                        [left + i, top + i, right - i, bottom - i],
                        outline=(255,0,0,125) #transperency 125 (1255)
                    )
                del draw
                #img_bbox = Image.alpha_composite(img_bbox,rect)
                img_bbox = np.asarray(img_bbox)

                # modify DICOM tags
                ds.PhotometricInterpretation = 'RGB'
                ds.SamplesPerPixel = 3
                ds.BitsAllocated = 8
                ds.BitsStored = 8
                ds.HighBit = 7
                ds.SeriesNumber = 50
                ds.add_new(0x00280006, 'US', 0)
                ds.is_little_endian = True
                ds.fix_meta_info()

                # save pixel data and dicom file
                ds.PixelData = img_bbox.tobytes()
                ds.save_as('dicom_nodules/'+str(count)+'.dcm')
                ds = dicom.read_file('dicom_nodules/'+str(count)+'.dcm')
                pixels = get_pixels_hu([ds])
                image = pixels
                img = image[0]
    else:
        img_bbox = Image.fromarray((255*normalized_image).astype('uint8'))
        img_bbox = img_bbox.convert('RGB')
        img_bbox = np.asarray(img_bbox)

        # modify DICOM tags
        ds.PhotometricInterpretation = 'RGB'
        ds.SamplesPerPixel = 3
        ds.BitsAllocated = 8
        ds.BitsStored = 8
        ds.HighBit = 7
        ds.SeriesNumber = 50
        ds.add_new(0x00280006, 'US', 0)
        ds.is_little_endian = True
        ds.fix_meta_info()

        # save pixel data and dicom file
        ds.PixelData = img_bbox.tobytes()
        ds.save_as('dicom_nodules/'+str(count)+'.dcm')

In [30]:
top, left, bottom, right = [211,99,291,158]
thickness = 4
img_bbox = Image.fromarray((255*display_img).astype('uint8'))
img_bbox = img_bbox.convert('RGB')
draw = ImageDraw.Draw(img_bbox)
for i in range(thickness):
    draw.rectangle(
        [left + i, top + i, right - i, bottom - i],
        outline=(255,0,0)
    )
del draw
img_bbox = np.asarray(img_bbox)

# modify DICOM tags
ds.PhotometricInterpretation = 'RGB'
ds.SamplesPerPixel = 3
ds.BitsAllocated = 8
ds.BitsStored = 8
ds.HighBit = 7
ds.add_new(0x00280006, 'US', 0)
ds.is_little_endian = True
ds.fix_meta_info()

# save pixel data and dicom file
ds.PixelData = img_bbox.tobytes()
ds.save_as('test.dcm')